In [ ]:
import pydantic
import sys
from pydantic import BaseModel
from datasets import load_dataset
import asyncio
import polars as pl
from IPython.display import Markdown


sys.path.append("..")
from synthetic_data.generation import RemoteModel, get_generation_wrapper

%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, PLAYWRIGHT_CONFIG, SMOL_LM_135M

dataset = load_dataset(
    "sam-paech/gutenberg3-generalfiction-scifi-fantasy-romance-adventure-dpo",
    trust_remote_code=True,
)["train"]
dataset_pl: pl.DataFrame = dataset.to_polars()


In [ ]:
dataset_pl.columns

In [ ]:
import re
for i, row in enumerate(dataset_pl.head(1000).gather_every(10).iter_rows(named=True)):
    source = row["source"]
    display(Markdown(f"### Sample {i} - {source}"))
    row_processed = _process_row(row)

In [47]:
from synthetic_data.generation import GenWrapperArgs
from typing import List
from enum import Enum
import json

model: str = RemoteModel.GPT_4O_MINI.value
generation_wrapper = get_generation_wrapper(
    model,
    GenWrapperArgs(model_id="gpt-4o-mini", response_format=Output),  # type: ignore
)  # type: ignore

sample = format_conversation_oai(_process_row(dataset_pl.head(1).to_dicts()[0]))

completions = await generation_wrapper.generate([sample])
print(completions)

2025-02-01 17:18:40.304 | INFO     | synthetic_data.generation:generate:144 - Generating 1 requests with gpt-4o-mini, max concurrent: 8


['{"items":[{"type":"action","content":"Katharine reached the study, Mrs. Hilbery had already dipped her pen in the ink.","character":"Narrator"},{"type":"dialogue","content":"\\"Katharine,\\" she said, lifting it in the air, \\"I\'ve just made out such a queer, strange thing about your grandfather. I\'m three years and six months older than he was when he died. I couldn\'t very well have been his mother, but I might have been his elder sister, and that seems to me such a pleasant fancy. I\'m going to start quite fresh this morning, and get a lot done.\\"","character":"Mrs. Hilbery"},{"type":"action","content":"Katharine sat down at her own table, untied the bundle of old letters upon which she was working, smoothed them out absent-mindedly, and began to decipher the faded script.","character":"Narrator"},{"type":"action","content":"She looked across at her mother, to judge her mood.","character":"Narrator"},{"type":"action","content":"Mrs. Hilbery was raising round her the skies and t

In [48]:
out_json = json.loads(completions[0])
out = Output.model_validate(out_json)
out.items

[SceneElement(type=<SceneElementType.ACTION: 'action'>, content='Katharine reached the study, Mrs. Hilbery had already dipped her pen in the ink.', character='Narrator'),
 SceneElement(type=<SceneElementType.DIALOGUE: 'dialogue'>, content='"Katharine," she said, lifting it in the air, "I\'ve just made out such a queer, strange thing about your grandfather. I\'m three years and six months older than he was when he died. I couldn\'t very well have been his mother, but I might have been his elder sister, and that seems to me such a pleasant fancy. I\'m going to start quite fresh this morning, and get a lot done."', character='Mrs. Hilbery'),
 SceneElement(type=<SceneElementType.ACTION: 'action'>, content='Katharine sat down at her own table, untied the bundle of old letters upon which she was working, smoothed them out absent-mindedly, and began to decipher the faded script.', character='Narrator'),
 SceneElement(type=<SceneElementType.ACTION: 'action'>, content='She looked across at her 